# Colab work preparations
*Note: remember to change/check if paths are correct*

### Prepare workspace

In [ ]:
# remove possible leftovers and clear-clone the whole repo
%cd /content
%rm -rf ./machine_learning_ev_ice
!git clone https://github.com/stas420/machine_learning_ev_ice.git

# a git pull pro forma
%cd ./machine_learning_ev_ice
!git pull

# go to our main directory
%cd ./src

# check (and download, if needed) dependencies for the project
%pip install -r requirements.txt

### A few global variables for Colab

In [ ]:
from google.colab import files
files.view('./CONSTANTS.py')

### Mount Google Drive data

Dataset uploaded to Google Drive shall be mounted for further work, it will be in ```/content/drive``` directory

In [ ]:
%cd /content

from CONSTANTS import CONTENT_DRIVE_PATH
from google.colab import drive
import os

if not 'drive' in os.listdir():
  drive.mount(CONTENT_DRIVE_PATH)
else:
  print("drajw dajrektory alredi prezent")

---
# Preparing data

### Downloading database
If you do not have it, you may download the data with this script

In [ ]:
# ---- download_videos.py ----
%cd /content/machine_learning_ev_ice/src

from google.colab import files
files.view('./download_videos.py')

%run ./download_videos.py

### Extracting video frames

***Important!*** *Below code cell runs, if - and only if - the frames are not yet present!* This procedure takes quite some time, and should be already done on your personal device, not on the remote VM.

In [ ]:
# ---- extract_video_frames.py ----
%cd /content/machine_learning_ev_ice/src

from google.colab import files
files.view('./extract_video_frames.py')

%run ./extract_video_frames.py

### Data augmentation

This code will duplicate a few of the extracted frames and alter them, to make dataset a bit more interesting

In [ ]:
# ---- frames_augmentation.py ----
%cd /content/machine_learning_ev_ice/src

from google.colab import files
files.view('./frames_augmentation.py')

%run ./frames_augmentation.py

***HERE LABEL THE IMAGES LOCALLY!***

### XML to YOLO format conversion

Labels made in labelImg are in XML format, which is not the same as YOLO labels format - cast is needed.

In [ ]:
# ---- convert_xml_to_yolo.py ----
%cd /content/machine_learning_ev_ice/src

from google.colab import files
files.view('./convert_xml_to_yolo.py')

%run ./convert_xml_to_yolo.py

### Move prepared data to training_data

In [ ]:
# ---- move_to_training_data.py ----
%cd /content/machine_learning_ev_ice/src

from google.colab import files
files.view('./move_to_training_data.py')

%run ./move_to_training_data.py

---

# Real work here

### Training the model with povided labeled data

The model first needs to understand what cars are EV, what cars are ICE, and what are "other" (i.e. indistinguishable, or vintage, possibly).

### Important check

Before proceeding, make sure there is a directory structured like this:

```
training_data/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```

and the files these folders contain have unique pairs (non-overlapping across train/val, too!) of corresponding names, i.e.:

images/train/img1.png <-> labels/traing/img1.txt; images/val/img2.png <-> labels/val/img2.png

### Configuration file

The *config.yaml* file configures the flow of the model training process, it looks like this:

```
# data.yaml
path: training_data

train: images/train
val: images/val

nc: 2
names:
  0: EV
  1: ICE
```

For full info, check out the [ultralytics docs](https://docs.ultralytics.com/usage/cfg/#predict-settings).

In [ ]:
################################################################
# Change this script in accordance to your working environment #
################################################################

%cd /content/machine_learning_ev_ice/src

from ultralytics import YOLO
import CONSTANTS

MODEL_PATH = './yolov8s.pt'
CONF_FILE = './config.yaml'
EPOCHS_NUM = 100

model = YOLO(MODEL_PATH)
model.train(data = CONF_FILE, epochs = EPOCHS_NUM)

---

# Validation helper

The below script helps you check whether the model is working ok by drawing the bounding boxes and writing assigned labels.

In [ ]:
# ---- validation_helper.py ----
%cd /content/machine_learning_ev_ice/src

from google.colab import files
files.view('./validation_helper.py')

%run ./validation_helper.py